In [3]:
%run ./data_creation_utils.ipynb

D:\KISHORE\Binance-Data-Downloader\data\downloaded_data
D:\KISHORE\Binance-Data-Downloader\data\extracted_data
['1000BONKUSDT', '1000BTTCUSDT', '1000FLOKIUSDT', '1000LUNCBUSD', '1000LUNCUSDT', '1000PEPEUSDT', '1000RATSUSDT', '1000SATSUSDT', '1000SHIBBUSD', '1000SHIBUSDT', '1000XECUSDT', '1INCHUSDT', 'AAVEUSDT', 'ACEUSDT', 'ACHUSDT', 'ADABUSD', 'ADAUSDT', 'AGIXBUSD', 'AGIXUSDT', 'AGLDUSDT', 'AKROUSDT', 'ALGOUSDT', 'ALICEUSDT', 'ALPHAUSDT', 'AMBBUSD', 'AMBUSDT', 'ANCBUSD', 'ANCUSDT', 'ANKRUSDT', 'ANTUSDT', 'APEBUSD', 'APEUSDT', 'API3USDT', 'APTBUSD', 'APTUSDT', 'ARBUSDT', 'ARKMUSDT', 'ARKUSDT', 'ARPAUSDT', 'ARUSDT', 'ASTRUSDT', 'ATAUSDT', 'ATOMUSDT', 'AUCTIONBUSD', 'AUCTIONUSDT', 'AUDIOUSDT', 'AVAXBUSD', 'AVAXUSDT', 'AXSUSDT', 'BADGERUSDT', 'BAKEUSDT', 'BALUSDT', 'BANDUSDT', 'BATUSDT', 'BCHUSDT', 'BEAMXUSDT', 'BELUSDT', 'BICOUSDT', 'BIGTIMEUSDT', 'BLUEBIRDUSDT', 'BLURUSDT', 'BLZUSDT', 'BNBBUSD', 'BNBUSDT', 'BNTUSDT', 'BNXUSDT', 'BNXUSDTSETTLED', 'BONDUSDT', 'BSVUSDT', 'BTCBUSD', 'BTCBUSD

data creation utilities successfully initialized


In [4]:
from pathlib import Path
import os
import re
import shutil
import talib
from datetime import *
import glob
import zipfile
import pandas as pd
from pandas.api.types import is_numeric_dtype
import inspect
import talib
import time
import numpy as np
import requests
import urllib.request


In [6]:


master_dictionary = {
    "symbols": [
#             SYMBOL_ARRAY[SYMBOL_ARRAY.index('BTCUSDT')],
        #     SYMBOL_ARRAY[SYMBOL_ARRAY.index('ETHUSDT')],
        #         SYMBOL_ARRAY[SYMBOL_ARRAY.index('ETHBUSD')],
#                 SYMBOL_ARRAY[SYMBOL_ARRAY.index('1000PEPEUSDT')],
        SYMBOL_ARRAY[SYMBOL_ARRAY.index('PYTHUSDT')],
        #         SYMBOL_ARRAY[SYMBOL_ARRAY.index('BTCBUSD')]
    ],
    "chart_times": [
#             CHART_TIME_ARRAY[CHART_TIME_ARRAY.index('5m')],
#             CHART_TIME_ARRAY[CHART_TIME_ARRAY.index('1m')],
#         CHART_TIME_ARRAY[CHART_TIME_ARRAY.index('15m')],
        CHART_TIME_ARRAY[CHART_TIME_ARRAY.index('4h')]
    ],
    "timeperiods": [5, 8, 13, 21, 30, 34, 50, 55, 89, 100, 144, 200, 233],
    "win_percentage":
    0.73,
    "loss_percentage":
    0.4,
    "monthly_or_daily_1_or_2":
    2
}

In [40]:
def download_data_and_concatenate(
        master_dictionary,
        win_perc=master_dictionary["win_percentage"],
        loss_perc=master_dictionary["loss_percentage"]):
    # Set the start date and end date
    start_date = datetime(2020, 1, 1)
    end_date = datetime(2080, 1, 1)

    # Initialize the list of year-month pairs
    MONTH_ARRAY = []

    # Iterate over the dates from start to end
    date = start_date
    while date < end_date:
        # Format the date as a year-month pair
        year_month = date.strftime("%Y-%m")
        #
        # Add the year-month pair to the list
        MONTH_ARRAY.append(year_month)

        # Increment the date by one month
        date += timedelta(days=31)
    # Get the current date
    now = datetime.now()

    # Format the date as a string in the desired format
    date_string = now.strftime("%Y-%m")

    idx = MONTH_ARRAY.index(date_string)
    MONTH_ARRAY = MONTH_ARRAY[:idx + 1]

    # Get the current date
    now = datetime.now()

    # Get the first day of the current month
    first_day = now.replace(day=1)

    # Create an empty list to store the days
    DAY_ARRAY = []

    # Loop through the days from the first day of the current month to today
    while first_day <= now:
        # Format the date as a string in the desired format
        date_string = first_day.strftime("%Y-%m-%d")

        # Add the date string to the list
        DAY_ARRAY.append(date_string)

        # Move to the next day
        first_day += timedelta(days=1)
   #     ####ONLY FOR TESTING PURPOSES######

    MONTH_ARRAY = [
        '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06',
        '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12',
        '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06',
        '2021-07', '2021-08', '2021-09', '2021-10', '2021-11', '2021-12',
        '2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06',
        '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12',
        '2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06',
        '2023-07', '2023-08', '2023-09'
    ]
    DAY_ARRAY = ['2023-10-01', '2023-10-02']
    #     ####ONLY FOR TESTING PURPOSES######

    for symbol in master_dictionary["symbols"]:
        for chart_time in master_dictionary["chart_times"]:
            print(f"setting up things for {symbol},{chart_time}")

            # Set up an empty list for the data frames
            df_list = []

            # Compile the regular expression pattern
            pattern = re.compile(
                f"^{symbol}-{chart_time}-\d{{4}}-\d{{2}}\.zip$")

            # Compile the regular expression pattern for daily zip files
            pattern_daily = re.compile(
                f"^{symbol}-{chart_time}-\d{{4}}-\d{{2}}-\d{{2}}\.zip$")

            # Create the new folder path for daily ZIP files
            new_daily_zip_folder_path = os.path.join(
                download_dir, f"{symbol}-{chart_time}-daily_data")

            # Create the new folder path for ZIP files
            new_zip_folder_path = os.path.join(
                download_dir, f"{symbol}-{chart_time}-monthly_data")

            # Create the new folder path for CSV files
            new_csv_folder_path = os.path.join(output_dir,
                                               f"{symbol}-{chart_time}")

            # Set the file name
            concatenated_file_name = f"{symbol}-{chart_time}_W{win_perc}_L{loss_perc}.csv"

            # Construct the file path
            concatenated_file_path = os.path.join(new_csv_folder_path,
                                                  concatenated_file_name)

            if not Path(concatenated_file_path).exists():
                print(
                    f"setting up things for the new symbol-chart_time combination : {symbol}{chart_time}"
                )
                download_monthly_data(month_array=MONTH_ARRAY,
                                      symbol=symbol,
                                      chart_time=chart_time)
                download_daily_data(day_array=DAY_ARRAY,
                                    symbol=symbol,
                                    chart_time=chart_time)

                # Iterate over the files in the new zip folder
                for file in os.listdir(new_zip_folder_path):
                    # Check if the file matches the pattern
                    if pattern.match(file):
                        # Construct the file path
                        file_path = os.path.join(new_zip_folder_path, file)

                        # Extract the ZIP file
                        with zipfile.ZipFile(file_path, "r") as zip_ref:
                            zip_ref.extractall(new_csv_folder_path)

                        # Construct the CSV file path
                        csv_file_path = os.path.join(
                            new_csv_folder_path,
                            f"{symbol}-{chart_time}{file[-12:-4]}.csv")

                        # Read the CSV file into a data frame, ignoring the headers
                        df = pd.read_csv(csv_file_path, header=None)

                        # Remove the first row (which contains the header)
                        df = df.iloc[1:]

                        # Add it to the list
                        df_list.append(df)

                # Iterate over the files in the new daily zip folder
                for file in os.listdir(new_daily_zip_folder_path):
                    # Check if the file matches the pattern
                    if pattern_daily.match(file):
                        # Construct the file path
                        file_path = os.path.join(new_daily_zip_folder_path,
                                                 file)

                        # Extract the ZIP file
                        with zipfile.ZipFile(file_path, "r") as zip_ref:
                            zip_ref.extractall(new_csv_folder_path)
                        # Construct the CSV file path
                        csv_file_path = os.path.join(
                            new_csv_folder_path,
                            f"{symbol}-{chart_time}-{file.split('-')[-3]}-{file.split('-')[-2]}-{file.split('-')[-1][:-4]}.csv"
                        )
                        # Read the CSV file into a data frame, ignoring the headers
                        df = pd.read_csv(csv_file_path, header=None)

                        # Remove the first row (which contains the header)
                        df = df.iloc[1:]

                        # Add it to the list
                        df_list.append(df)
                # Concatenate the data frames in the list
                df_final = pd.concat(df_list, ignore_index=True)

                # Read the headers from the first CSV file
                headers = pd.read_csv(csv_file_path, nrows=1).columns

                # Set the headers as the column names of the final dataframe
                df_final.columns = headers

                # Convert 'open_time' and 'close_time' columns to datetime
                df_final['open_time'] = pd.to_datetime(
                    df_final['open_time'], unit='ms').dt.tz_localize(
                        'UTC').dt.tz_convert('Asia/Kolkata')

                # Delete the 'ignore' column
                df_final = df_final.drop(['ignore', 'close_time'], axis=1)

                # Add a new column called 'entry' that will take open of candle
                df_final['entry'] = df_final['open']

                # Write the data frame to the Excel file
                df_final.to_csv(concatenated_file_path, index=False)
                
                directory_final = Path(
                    concatenated_file_path).parent  # Get the parent directory

                # deleting all the other csvs
                for file_path in directory_final.iterdir():
                    if file_path != Path(concatenated_file_path):
                        if file_path.is_file():
                            file_path.unlink()
                calculate_wins_losses(master_dictionary=master_dictionary,
                      win_perc=master_dictionary["win_percentage"],
                      loss_perc=master_dictionary["loss_percentage"]
                     )

                calculate_indicator_values_new(
                    master_dictionary=master_dictionary,
                    monthly_or_daily_1_2=master_dictionary["monthly_or_daily_1_or_2"],
                    win_perc=master_dictionary["win_percentage"],
                    loss_perc=master_dictionary["loss_percentage"])
                return f"things set up for the new symbol-chart_time combination : {symbol}{chart_time}"
            else:
                print(
    f"setting up things only for incremental data for {symbol}{chart_time}"
                )
                 # Set up an empty list for the data frames
                df_list = []

                # Compile the regular expression pattern
                pattern = re.compile(
                    f"^{symbol}-{chart_time}-\d{{4}}-\d{{2}}\.zip$")

                # Compile the regular expression pattern for daily zip files
                pattern_daily = re.compile(
                    f"^{symbol}-{chart_time}-\d{{4}}-\d{{2}}-\d{{2}}\.zip$")

                # Create the new folder path for daily ZIP files
                new_daily_zip_folder_path = os.path.join(
                    download_dir, f"{symbol}-{chart_time}-daily_data")

                # Create the new folder path for ZIP files
                new_zip_folder_path = os.path.join(
                    download_dir, f"{symbol}-{chart_time}-monthly_data")

                # Create the new folder path for CSV files
                new_csv_folder_path = os.path.join(output_dir,
                                                   f"{symbol}-{chart_time}")

                # Set the file name
                concatenated_file_name = f"{symbol}-{chart_time}_W{win_perc}_L{loss_perc}.csv"

                # Construct the file path
                concatenated_file_path = os.path.join(new_csv_folder_path,
                                                      concatenated_file_name)

                df = pd.read_csv(concatenated_file_path, header=None)
                last_500_records = df.iloc[1:,0:11] #earlier it was df.iloc[-500:,0:11] but 500 records is throwing some sort of issue so using the df completely

                df_list.append(last_500_records)

                df = pd.read_csv(concatenated_file_path)
                df['open_time'] = pd.to_datetime(df['open_time'])
                last_record = df.iloc[-1:]

                def calculate_reference_time(chart_time, base_time):
                    # Extract the numeric value and unit from chart_time
                    numeric_value = int(chart_time[:-1])
                    unit = chart_time[-1]

                    # Define a dictionary to map units to timedelta objects
                    time_units = {
                        'm': timedelta(minutes=numeric_value),
                        'h': timedelta(hours=numeric_value),
                    }

                    # Calculate the reference time based on the unit and numeric value
                    reference_time = base_time - time_units[unit]
                    return reference_time.strftime('%H:%M:%S%z')

                base_time = datetime.strptime("05:30:00+0530", '%H:%M:%S%z')
                reference_time = calculate_reference_time(
                    chart_time, base_time)
                print(reference_time)
                # Check if the time part of 'open_time' is "05:15:00+05:30"


                if last_record['open_time'].dt.strftime(
                        '%H:%M:%S%z').item() == reference_time:
                    # Calculate the last processed date by subtracting 1 day from the date part
                    last_processed_date = (
                        last_record['open_time'].dt.date -
                        pd.DateOffset(days=1)).item().strftime('%Y-%m-%d')
                    last_processed_date = datetime.strptime(
                        last_processed_date, "%Y-%m-%d")
                    print(last_processed_date)
                    # Get today's date
                    today = datetime.today()
                    print(today)

                    # Initialize DAY_ARRAY
                    DAY_ARRAY = []
                    # Start from the day after last_processed_date
                    current_day = last_processed_date + timedelta(days=1)

                    while current_day <= today:
                        # Format the date as a string in the desired format
                        date_string = current_day.strftime("%Y-%m-%d")

                        # Add the date string to DAY_ARRAY
                        DAY_ARRAY.append(date_string)

                        # Move to the next day
                        current_day += timedelta(days=1)
                    print(DAY_ARRAY)
                    download_daily_data(day_array=DAY_ARRAY,
                                        symbol=symbol,
                                        chart_time=chart_time)
                    # Iterate over the files in the new daily zip folder
                    for file in Path(new_daily_zip_folder_path).iterdir():
                        # Check if the file matches the pattern
                        if pattern_daily.match(file.name):
                            # Extract the date part from the file name (e.g., "2023-08-19")
                            zip_date = file.name.split('-')[-3:]
                            zip_date_str = '-'.join(zip_date).replace(
                                '.zip', '')

                            # Check if the date is in DAY_ARRAY
                            if zip_date_str in DAY_ARRAY:
                                # Construct the file path
                                file_path = file

                                # Extract the ZIP file
                                with zipfile.ZipFile(file_path,
                                                     "r") as zip_ref:
                                    zip_ref.extractall(new_csv_folder_path)

                                # Construct the CSV file path
                                csv_file_path = Path(
                                    new_csv_folder_path
                                ) / f"{symbol}-{chart_time}-{zip_date_str}.csv"

                                # Read the CSV file into a data frame, ignoring the headers
                                df = pd.read_csv(csv_file_path, header=None)

                                # Remove the first row (which contains the header)
                                df = df.iloc[1:]

                                # Add it to the list
                                df_list.append(df)
                    df_list[0].columns = [0,1,2,3,4,5,7,8,9,10,11]
                    for df in df_list[1:]:
                        # Convert 'open_time' column to datetime
                        df[0] = pd.to_datetime(
                                            df[0], unit='ms').dt.tz_localize(
                                                'UTC').dt.tz_convert('Asia/Kolkata')
                        cols_to_drop = [6, 11]
                        df.drop(df.columns[cols_to_drop], axis=1, inplace=True)
                        df[11] = df[1]
                #         display(df.head())

                    df_final = pd.concat(df_list[:], axis=0, ignore_index=True)
                #     df_final.drop(df_final.columns[11], axis=1,inplace=True)
                    # Read the headers from the first CSV file
                    headers = pd.read_csv(concatenated_file_path, nrows=1, usecols=range(11)).columns

                    print(headers)
                    # Set the headers as the column names of the final dataframe
                    df_final.columns = headers[0:11]
                    df = df_final
                    df.reset_index(drop=True, inplace=True)
                    display(df)
                    df["open"] = pd.to_numeric(df["open"])
                    df["high"] = pd.to_numeric(df["high"])
                    df["low"] = pd.to_numeric(df["low"])
                    df["close"] = pd.to_numeric(df["close"])
                    df["entry"] = pd.to_numeric(df["entry"])
                    print(f"calculating win/loss for incremental data for {symbol}{chart_time} winning at {win_perc} and losing at {loss_perc}")
                    # Construct the file name
                    df["if_short"] = 0
                    df["if_long"] = 0
                    df["long_target"] = np.nan
                    df["short_target"] = np.nan
                    df["long_stop_loss"] = np.nan
                    df["short_stop_loss"] = np.nan
                    df["shorts_win_after"] = np.nan
                    df["longs_win_after"] = np.nan
                    df["dual_loss"] = 0
                    df["entered_before"] = np.nan

                    for i in range(len(df)):
                        if df["entry"][i]:
                            long_target = df["entry"][i] * (1 + win_perc / 100)
                            short_target = df["entry"][i] * (1 - win_perc / 100)
                            long_stop_loss = df["entry"][i] * (1 - loss_perc / 100)
                            short_stop_loss = df["entry"][i] * (1 + loss_perc / 100)
                            df.loc[i, 'long_target'] = long_target
                            df.loc[i, 'long_stop_loss'] = long_stop_loss
                            df.loc[i, 'short_target'] = short_target
                            df.loc[i, 'short_stop_loss'] = short_stop_loss
                            for j in range(i, len(df)):
                                if df["high"][j] >= long_target:
                                    if df["low"][j] <= long_stop_loss:
                                        df.loc[i, 'if_long'] = -1
                                        df.loc[i, 'dual_loss'] = 1
                                        df.loc[i, 'entered_before'] = j - i
                                    else:
                                        df.loc[i, 'if_long'] = 1
                                        df.loc[i, 'longs_win_after'] = j - i
                                    break
                                elif df["low"][j] <= long_stop_loss:
                                    df.loc[i, 'if_long'] = -1
                                    break
                            for j in range(i, len(df)):
                                if df["low"][j] <= short_target:
                                    if df["high"][j] >= short_stop_loss:
                                        df.loc[i, 'if_short'] = -1
                                        df.loc[i, 'dual_loss'] = 1
                                        df.loc[i, 'entered_before'] = j - i
                                    else:
                                        df.loc[i, 'if_short'] = 1
                                        df.loc[i, 'shorts_win_after'] = j - i
                                    break
                                elif df["high"][j] >= short_stop_loss:
                                    df.loc[i, 'if_short'] = -1
                                    break

                    new_columns = pd.DataFrame()
                    new_columns['HT_TRENDLINE'] = talib.HT_TRENDLINE(df['close'])
                    #         df['MAMA'], df['FAMA'] = talib.MAMA(df['close'], fastlimit=0, slowlimit=0)
                    #         df['MAVP'] = talib.MAVP(df['close'], periods=None, minperiod=2, maxperiod=30, matype=0)
                    new_columns['SAR'] = talib.SAR(df['high'], df['low'], acceleration=0, maximum=0)
                    new_columns['SAREXT'] = talib.SAREXT(df['high'], df['low'])
                    new_columns['T3'] = talib.T3(df['close'], timeperiod=5, vfactor=0)
                #             #########Momentum Indicators
                    new_columns['APO'] = talib.APO(df['close'], fastperiod=12, slowperiod=26)
                    new_columns['BOP'] = talib.BOP(df['open'], df['high'], df['low'], df['close'])
                    macd, macd_signal, macd_hist = talib.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)
                    new_columns['MACD'] = macd
                    new_columns['MACD_signal'] = macd_signal
                    new_columns['MACD_hist'] = macd_hist
                    new_columns['PPO'] = talib.PPO(df['close'], fastperiod=12, slowperiod=26, matype=0)
                    new_columns['TRIX'] = talib.TRIX(df['close'])
                    new_columns['ULTOSC'] = talib.ULTOSC(df['high'], df['low'], df['close'])
                    new_columns['WILLR'] = talib.WILLR(df['high'], df['low'], df['close'])



                    # Not Working ATM
                    #         new_columns['STOCH'] = talib.STOCH(df['high'], df['low'], df['close'])
                    #         new_columns['STOCHF'] = talib.STOCHF(df['high'], df['low'], df['close'])
                    #         new_columns['STOCHRSI'] = talib.STOCHRSI(df['close'])
                    #         new_columns['MACDEXT'] = talib.MACDEXT(df['close'], fastperiod=12, fastmatype=0, slowperiod=26, slowmatype=0, signalperiod=9, signalmatype=0)
                    #         new_columns['MACDFIX'] = talib.MACDFIX(df['close'], signalperiod=9)
                    #########Volume Indicators
                    new_columns['AD'] = talib.AD(df['high'], df['low'], df['close'], df['volume'])
                    new_columns['ADOSC'] = talib.ADOSC(df['high'], df['low'], df['close'], df['volume'], fastperiod=3, slowperiod=10)
                    new_columns['OBV'] = talib.OBV(df['close'], df['volume'])

                    #########Cycle Indicators
                    new_columns['HT_DCPERIOD'] = talib.HT_DCPERIOD(df['close'])
                    new_columns['HT_DCPHASE'] = talib.HT_DCPHASE(df['close'])
                    new_columns['HT_PHASOR_inphase'], new_columns['HT_PHASOR_quadrature'] = talib.HT_PHASOR(df['close'])
                    # new_columns['HT_SINE'] = talib.HT_SINE(df['close'])
                    new_columns['HT_TRENDMODE'] = talib.HT_TRENDMODE(df['close'])

                    #########Price Transform
                    new_columns['AVGPRICE'] = talib.AVGPRICE(df['open'], df['high'], df['low'], df['close'])
                    new_columns['MEDPRICE'] = talib.MEDPRICE(df['high'], df['low'])
                    new_columns['TYPPRICE'] = talib.TYPPRICE(df['high'], df['low'], df['close'])
                    new_columns['WCLPRICE'] = talib.WCLPRICE(df['high'], df['low'], df['close'])
                    #########Volatility Indicators
                    new_columns['TRANGE'] = talib.TRANGE(df['high'], df['low'], df['close'])
                    #########Pattern Recognition
                    new_columns['CDL2CROWS'] = talib.CDL2CROWS(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDL3BLACKCROWS'] = talib.CDL3BLACKCROWS(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDL3INSIDE'] = talib.CDL3INSIDE(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDL3LINESTRIKE'] = talib.CDL3LINESTRIKE(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDL3OUTSIDE'] = talib.CDL3OUTSIDE(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDL3STARSINSOUTH'] = talib.CDL3STARSINSOUTH(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDL3WHITESOLDIERS'] = talib.CDL3WHITESOLDIERS(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLABANDONEDBABY'] = talib.CDLABANDONEDBABY(df['open'], df['high'], df['low'], df['close'], penetration=0)

                    new_columns['CDLADVANCEBLOCK'] = talib.CDLADVANCEBLOCK(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLBELTHOLD'] = talib.CDLBELTHOLD(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLBREAKAWAY'] = talib.CDLBREAKAWAY(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLCLOSINGMARUBOZU'] = talib.CDLCLOSINGMARUBOZU(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLCONCEALBABYSWALL'] = talib.CDLCONCEALBABYSWALL(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLCOUNTERATTACK'] = talib.CDLCOUNTERATTACK(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLDARKCLOUDCOVER'] = talib.CDLDARKCLOUDCOVER(df['open'], df['high'], df['low'], df['close'], penetration=0)

                    new_columns['CDLDOJI'] = talib.CDLDOJI(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLDOJISTAR'] = talib.CDLDOJISTAR(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLDRAGONFLYDOJI'] = talib.CDLDRAGONFLYDOJI(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLENGULFING'] = talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLEVENINGDOJISTAR'] = talib.CDLEVENINGDOJISTAR(df['open'], df['high'], df['low'], df['close'], penetration=0)

                    new_columns['CDLEVENINGSTAR'] = talib.CDLEVENINGSTAR(df['open'], df['high'], df['low'], df['close'], penetration=0)
                    new_columns['CDLGAPSIDESIDEWHITE'] = talib.CDLGAPSIDESIDEWHITE(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLGRAVESTONEDOJI'] = talib.CDLGRAVESTONEDOJI(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLHAMMER'] = talib.CDLHAMMER(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLHANGINGMAN'] = talib.CDLHANGINGMAN(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLHARAMI'] = talib.CDLHARAMI(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLHARAMICROSS'] = talib.CDLHARAMICROSS(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLHIGHWAVE'] = talib.CDLHIGHWAVE(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLHIKKAKE'] = talib.CDLHIKKAKE(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLHIKKAKEMOD'] = talib.CDLHIKKAKEMOD(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLHOMINGPIGEON'] = talib.CDLHOMINGPIGEON(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLIDENTICAL3CROWS'] = talib.CDLIDENTICAL3CROWS(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLINNECK'] = talib.CDLINNECK(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLINVERTEDHAMMER'] = talib.CDLINVERTEDHAMMER(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLKICKING'] = talib.CDLKICKING(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLKICKINGBYLENGTH'] = talib.CDLKICKINGBYLENGTH(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLLADDERBOTTOM'] = talib.CDLLADDERBOTTOM(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLLONGLEGGEDDOJI'] = talib.CDLLONGLEGGEDDOJI(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLLONGLINE'] = talib.CDLLONGLINE(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLMARUBOZU'] = talib.CDLMARUBOZU(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLMATCHINGLOW'] = talib.CDLMATCHINGLOW(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLMATHOLD'] = talib.CDLMATHOLD(df['open'], df['high'], df['low'], df['close'], penetration=0)
                    new_columns['CDLMORNINGDOJISTAR'] = talib.CDLMORNINGDOJISTAR(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLMORNINGSTAR'] = talib.CDLMORNINGSTAR(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLONNECK'] = talib.CDLONNECK(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLPIERCING'] = talib.CDLPIERCING(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLRICKSHAWMAN'] = talib.CDLRICKSHAWMAN(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLRISEFALL3METHODS'] = talib.CDLRISEFALL3METHODS(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLSEPARATINGLINES'] = talib.CDLSEPARATINGLINES(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLSHOOTINGSTAR'] = talib.CDLSHOOTINGSTAR(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLSHORTLINE'] = talib.CDLSHORTLINE(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLSPINNINGTOP'] = talib.CDLSPINNINGTOP(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLSTALLEDPATTERN'] = talib.CDLSTALLEDPATTERN(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLSTICKSANDWICH'] = talib.CDLSTICKSANDWICH(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLTAKURI'] = talib.CDLTAKURI(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLTASUKIGAP'] = talib.CDLTASUKIGAP(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLTHRUSTING'] = talib.CDLTHRUSTING(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLTRISTAR'] = talib.CDLTRISTAR(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLUNIQUE3RIVER'] = talib.CDLUNIQUE3RIVER(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLUPSIDEGAP2CROWS'] = talib.CDLUPSIDEGAP2CROWS(df['open'], df['high'], df['low'], df['close'])
                    new_columns['CDLXSIDEGAP3METHODS'] = talib.CDLXSIDEGAP3METHODS(df['open'], df['high'], df['low'], df['close'])
                    #########Statistic Functions
                    new_columns['LINEARREG'] = talib.LINEARREG(df['close'])
                    new_columns['LINEARREG_ANGLE'] = talib.LINEARREG_ANGLE(df['close'])
                    new_columns['LINEARREG_INTERCEPT'] = talib.LINEARREG_INTERCEPT(df['close'])
                    new_columns['LINEARREG_SLOPE'] = talib.LINEARREG_SLOPE(df['close'])
                    #         new_columns['STDDEV'] = df['close'].rolling(timeperiod).std()
                    new_columns['TSF'] = talib.TSF(df['close'])
                    new_columns['VAR'] = talib.VAR(df['close'])
                    # Iterate over the time periods
                    for timeperiod in master_dictionary["timeperiods"]:
                        #########Overlap Studies
                        new_columns[f'BB_upper_{timeperiod}'], new_columns[f'BB_middle_{timeperiod}'], new_columns[
                            f'BB_lower_{timeperiod}'] = talib.BBANDS(df['close'], timeperiod=timeperiod)
                        new_columns[f'DEMA_{timeperiod}'] = talib.DEMA(df['close'], timeperiod=timeperiod)
                        new_columns[f'EMA_{timeperiod}'] = talib.EMA(df['close'], timeperiod=timeperiod)
                        new_columns[f'KAMA_{timeperiod}'] = talib.KAMA(df['close'], timeperiod=timeperiod)
                        new_columns[f'MA_{timeperiod}'] = talib.MA(df['close'], timeperiod=timeperiod)
                        #         new_columns['MAMA'], new_columns['FAMA'] = talib.MAMA(df['close'], fastlimit=0, slowlimit=0)
                        #         new_columns['MAVP'] = talib.MAVP(df['close'], periods=None, minperiod=2, maxperiod=30, matype=0)
                        new_columns[f'MIDPOINT_{timeperiod}'] = talib.MIDPOINT(df['close'], timeperiod=timeperiod)
                        new_columns[f'MIDPRICE_{timeperiod}'] = talib.MIDPRICE(df['high'], df['low'], timeperiod=timeperiod)
                        new_columns[f'SMA_{timeperiod}'] = talib.SMA(df['close'], timeperiod=timeperiod)
                        new_columns[f'TEMA_{timeperiod}'] = talib.TEMA(df['close'], timeperiod=timeperiod)
                        new_columns[f'TRIMA_{timeperiod}'] = talib.TRIMA(df['close'], timeperiod=timeperiod)
                        new_columns[f'WMA_{timeperiod}'] = talib.WMA(df['close'], timeperiod=timeperiod)
                        new_columns[f'ADX_{timeperiod}'] = talib.ADX(df['high'], df['low'], df['close'], timeperiod=timeperiod)
                        new_columns[f'ADXR_{timeperiod}'] = talib.ADXR(df['high'], df['low'], df['close'], timeperiod=timeperiod)
                        new_columns[f'AROON_up_{timeperiod}'], new_columns[f'AROON_down_{timeperiod}'] = talib.AROON(df['high'], df['low'],
                                                                                                   timeperiod=timeperiod)
                        new_columns[f'AROONOSC_{timeperiod}'] = talib.AROONOSC(df['high'], df['low'], timeperiod=timeperiod)
                        new_columns[f'CCI_{timeperiod}'] = talib.CCI(df['high'], df['low'], df['close'], timeperiod=timeperiod)
                        new_columns[f'CMO_{timeperiod}'] = talib.CMO(df['close'], timeperiod=timeperiod)
                        new_columns[f'DX_{timeperiod}'] = talib.DX(df['high'], df['low'], df['close'], timeperiod=timeperiod)
                        new_columns[f'MFI_{timeperiod}'] = talib.MFI(df['high'], df['low'], df['close'], df['volume'],
                                                            timeperiod=timeperiod)
                        new_columns[f'MINUS_DI_{timeperiod}'] = talib.MINUS_DI(df['high'], df['low'], df['close'], timeperiod=timeperiod)
                        new_columns[f'MINUS_DM_{timeperiod}'] = talib.MINUS_DM(df['high'], df['low'], timeperiod=timeperiod)
                        new_columns[f'MOM_{timeperiod}'] = talib.MOM(df['close'], timeperiod=timeperiod)
                        new_columns[f'PLUS_DI_{timeperiod}'] = talib.PLUS_DI(df['high'], df['low'], df['close'], timeperiod=timeperiod)
                        new_columns[f'PLUS_DM_{timeperiod}'] = talib.PLUS_DM(df['high'], df['low'], timeperiod=timeperiod)
                        new_columns[f'ROC_{timeperiod}'] = talib.ROC(df['close'], timeperiod=timeperiod)
                        new_columns[f'ROCP_{timeperiod}'] = talib.ROCP(df['close'], timeperiod=timeperiod)
                        new_columns[f'ROCR_{timeperiod}'] = talib.ROCR(df['close'], timeperiod=timeperiod)
                        new_columns[f'ROCR100_{timeperiod}'] = talib.ROCR100(df['close'], timeperiod=timeperiod)
                        new_columns[f'RSI_{timeperiod}'] = talib.RSI(df['close'], timeperiod=timeperiod)

                        new_columns[f'ATR_{timeperiod}'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=timeperiod)
                        new_columns[f'NATR_{timeperiod}'] = talib.NATR(df['high'], df['low'], df['close'], timeperiod=timeperiod)
                        #########Statistic Functions
                        new_columns[f'BETA_{timeperiod}'] = talib.BETA(df['high'], df['low'], timeperiod=timeperiod)
                        new_columns[f'CORREL_{timeperiod}'] = talib.CORREL(df['high'], df['low'], timeperiod=timeperiod)
                    # Save the updated dataframe to the CSV file
                    df = pd.concat([df, new_columns], axis=1)
                    df_final = df[len(last_500_records):]
                #     Append the data frame to the existing CSV file in append mode without headers
                    df_final.to_csv(concatenated_file_path,
                                    mode='a',
                                    header=False,
                                    index=False)
                    directory_final = Path(
                        concatenated_file_path
                    ).parent  # Get the parent directory

                    # deleting all the other csvs
                    for file_path in directory_final.iterdir():
                        if file_path != Path(concatenated_file_path):
                            if file_path.is_file():
                                file_path.unlink()
                    return f"things set up only for incremental data for {symbol}{chart_time}"
                else:
                    return f"Something is wrong with data. Last recorded time is not in sync with what is expected. Expected {reference_time} but got {last_record['open_time'].dt.strftime('%H:%M:%S%z').item()}"

In [8]:
download_data_and_concatenate(master_dictionary, MONTH_ARRAY, DAY_ARRAY)

NameError: name 'DAY_ARRAY' is not defined

## Calculating Wins/Losses and Indicators

In [ ]:
calculate_wins_losses(master_dictionary, win_perc=0.73, loss_perc=0.4)

In [ ]:
calculate_indicator_values(master_dictionary, win_perc=0.73, loss_perc=0.4)

## test 14-11-2023

In [23]:

concatenated_file_path = "D:\KISHORE\Binance-Data-Downloader\data\extracted_data\BTCUSDT-4h\BTCUSDT-4h_W0.73_L0.4.csv"

df = pd.read_csv(concatenated_file_path)
df['open_time'] = pd.to_datetime(df['open_time'])
last_record = df.iloc[-500:]
display(last_record)
print(len(last_record))


symbol = "BTCUSDT"
chart_time = "4h"
win_perc= 0.73
loss_perc = 0.4



print(
    f"setting up things only for incremental data for {symbol}{chart_time}"
)
 # Set up an empty list for the data frames
df_list = []

# Compile the regular expression pattern
pattern = re.compile(
    f"^{symbol}-{chart_time}-\d{{4}}-\d{{2}}\.zip$")

# Compile the regular expression pattern for daily zip files
pattern_daily = re.compile(
    f"^{symbol}-{chart_time}-\d{{4}}-\d{{2}}-\d{{2}}\.zip$")

# Create the new folder path for daily ZIP files
new_daily_zip_folder_path = os.path.join(
    download_dir, f"{symbol}-{chart_time}-daily_data")

# Create the new folder path for ZIP files
new_zip_folder_path = os.path.join(
    download_dir, f"{symbol}-{chart_time}-monthly_data")

# Create the new folder path for CSV files
new_csv_folder_path = os.path.join(output_dir,
                                   f"{symbol}-{chart_time}")

# Set the file name
concatenated_file_name = f"{symbol}-{chart_time}_W{win_perc}_L{loss_perc}.csv"

# Construct the file path
concatenated_file_path = os.path.join(new_csv_folder_path,
                                      concatenated_file_name)

df = pd.read_csv(concatenated_file_path, header=None)
last_2000_records = df.iloc[-2000:,0:11]

display(last_2000_records)
print(last_2000_records.dtypes)
df_list.append(last_2000_records)

df = pd.read_csv(concatenated_file_path)
df['open_time'] = pd.to_datetime(df['open_time'])
last_record = df.iloc[-1:]

def calculate_reference_time(chart_time, base_time):
    # Extract the numeric value and unit from chart_time
    numeric_value = int(chart_time[:-1])
    unit = chart_time[-1]

    # Define a dictionary to map units to timedelta objects
    time_units = {
        'm': timedelta(minutes=numeric_value),
        'h': timedelta(hours=numeric_value),
    }

    # Calculate the reference time based on the unit and numeric value
    reference_time = base_time - time_units[unit]
    return reference_time.strftime('%H:%M:%S%z')

base_time = datetime.strptime("05:30:00+0530", '%H:%M:%S%z')
reference_time = calculate_reference_time(
    chart_time, base_time)
print(reference_time)
# Check if the time part of 'open_time' is "05:15:00+05:30"


if last_record['open_time'].dt.strftime(
        '%H:%M:%S%z').item() == reference_time:
    # Calculate the last processed date by subtracting 1 day from the date part
    last_processed_date = (
        last_record['open_time'].dt.date -
        pd.DateOffset(days=1)).item().strftime('%Y-%m-%d')
    last_processed_date = datetime.strptime(
        last_processed_date, "%Y-%m-%d")
    print(last_processed_date)
    # Get today's date
    today = datetime.today()
    print(today)

    # Initialize DAY_ARRAY
    DAY_ARRAY = []
    # Start from the day after last_processed_date
    current_day = last_processed_date + timedelta(days=1)

    while current_day <= today:
        # Format the date as a string in the desired format
        date_string = current_day.strftime("%Y-%m-%d")

        # Add the date string to DAY_ARRAY
        DAY_ARRAY.append(date_string)

        # Move to the next day
        current_day += timedelta(days=1)
#     print(DAY_ARRAY)
#     download_daily_data(day_array=DAY_ARRAY,
#                         symbol=symbol,
#                         chart_time=chart_time)
    # Iterate over the files in the new daily zip folder
    for file in Path(new_daily_zip_folder_path).iterdir():
        # Check if the file matches the pattern
        if pattern_daily.match(file.name):
            # Extract the date part from the file name (e.g., "2023-08-19")
            zip_date = file.name.split('-')[-3:]
            zip_date_str = '-'.join(zip_date).replace(
                '.zip', '')

            # Check if the date is in DAY_ARRAY
            if zip_date_str in DAY_ARRAY:
                # Construct the file path
                file_path = file

                # Extract the ZIP file
                with zipfile.ZipFile(file_path,
                                     "r") as zip_ref:
                    zip_ref.extractall(new_csv_folder_path)

                # Construct the CSV file path
                csv_file_path = Path(
                    new_csv_folder_path
                ) / f"{symbol}-{chart_time}-{zip_date_str}.csv"

                # Read the CSV file into a data frame, ignoring the headers
                df = pd.read_csv(csv_file_path, header=None)

                # Remove the first row (which contains the header)
                df = df.iloc[1:]

                # Add it to the list
                df_list.append(df)
    df_list[0].columns = [0,1,2,3,4,5,7,8,9,10,11]
    display(df_list[0].head())
    display(df_list[1].head())
    for df in df_list[1:]:
        # Convert 'open_time' column to datetime
        df[0] = pd.to_datetime(
                            df[0], unit='ms').dt.tz_localize(
                                'UTC').dt.tz_convert('Asia/Kolkata')
        cols_to_drop = [6, 11]
        df.drop(df.columns[cols_to_drop], axis=1, inplace=True)
        df[11] = df[1]
#         display(df.head())
    
    df_final = pd.concat(df_list[:], axis=0, ignore_index=True)
#     df_final.drop(df_final.columns[11], axis=1,inplace=True)
#     display(df_final.head())
#     display(df_final[500:].head())
    # Read the headers from the first CSV file
    headers = pd.read_csv(concatenated_file_path, nrows=1, usecols=range(11)).columns

    print(headers)
    # Set the headers as the column names of the final dataframe
    df_final.columns = headers[0:11]
    print(df_final.columns)
    df = df_final
    df.reset_index(drop=True, inplace=True)
    display(df)
    df["open"] = pd.to_numeric(df["open"])
    df["high"] = pd.to_numeric(df["high"])
    df["low"] = pd.to_numeric(df["low"])
    df["close"] = pd.to_numeric(df["close"])
    df["entry"] = pd.to_numeric(df["entry"])
    print(f"calculating win/loss for incremental data for {symbol}{chart_time} winning at {win_perc} and losing at {loss_perc}")
    # Construct the file name
    df["if_short"] = 0
    df["if_long"] = 0
    df["long_target"] = np.nan
    df["short_target"] = np.nan
    df["long_stop_loss"] = np.nan
    df["short_stop_loss"] = np.nan
    df["shorts_win_after"] = np.nan
    df["longs_win_after"] = np.nan
    df["dual_loss"] = 0
    df["entered_before"] = np.nan

    for i in range(len(df)):
        if df["entry"][i]:
            long_target = df["entry"][i] * (1 + win_perc / 100)
            short_target = df["entry"][i] * (1 - win_perc / 100)
            long_stop_loss = df["entry"][i] * (1 - loss_perc / 100)
            short_stop_loss = df["entry"][i] * (1 + loss_perc / 100)
            df.loc[i, 'long_target'] = long_target
            df.loc[i, 'long_stop_loss'] = long_stop_loss
            df.loc[i, 'short_target'] = short_target
            df.loc[i, 'short_stop_loss'] = short_stop_loss
            for j in range(i, len(df)):
                if df["high"][j] >= long_target:
                    if df["low"][j] <= long_stop_loss:
                        df.loc[i, 'if_long'] = -1
                        df.loc[i, 'dual_loss'] = 1
                        df.loc[i, 'entered_before'] = j - i
                    else:
                        df.loc[i, 'if_long'] = 1
                        df.loc[i, 'longs_win_after'] = j - i
                    break
                elif df["low"][j] <= long_stop_loss:
                    df.loc[i, 'if_long'] = -1
                    break
            for j in range(i, len(df)):
                if df["low"][j] <= short_target:
                    if df["high"][j] >= short_stop_loss:
                        df.loc[i, 'if_short'] = -1
                        df.loc[i, 'dual_loss'] = 1
                        df.loc[i, 'entered_before'] = j - i
                    else:
                        df.loc[i, 'if_short'] = 1
                        df.loc[i, 'shorts_win_after'] = j - i
                    break
                elif df["high"][j] >= short_stop_loss:
                    df.loc[i, 'if_short'] = -1
                    break
    
    new_columns = pd.DataFrame()
    new_columns['HT_TRENDLINE'] = talib.HT_TRENDLINE(df['close'])
    #         df['MAMA'], df['FAMA'] = talib.MAMA(df['close'], fastlimit=0, slowlimit=0)
    #         df['MAVP'] = talib.MAVP(df['close'], periods=None, minperiod=2, maxperiod=30, matype=0)
    new_columns['SAR'] = talib.SAR(df['high'], df['low'], acceleration=0, maximum=0)
    new_columns['SAREXT'] = talib.SAREXT(df['high'], df['low'])
    new_columns['T3'] = talib.T3(df['close'], timeperiod=5, vfactor=0)
#             #########Momentum Indicators
    new_columns['APO'] = talib.APO(df['close'], fastperiod=12, slowperiod=26)
    new_columns['BOP'] = talib.BOP(df['open'], df['high'], df['low'], df['close'])
    macd, macd_signal, macd_hist = talib.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)
    new_columns['MACD'] = macd
    new_columns['MACD_signal'] = macd_signal
    new_columns['MACD_hist'] = macd_hist
    new_columns['PPO'] = talib.PPO(df['close'], fastperiod=12, slowperiod=26, matype=0)
    new_columns['TRIX'] = talib.TRIX(df['close'])
    new_columns['ULTOSC'] = talib.ULTOSC(df['high'], df['low'], df['close'])
    new_columns['WILLR'] = talib.WILLR(df['high'], df['low'], df['close'])



    # Not Working ATM
    #         new_columns['STOCH'] = talib.STOCH(df['high'], df['low'], df['close'])
    #         new_columns['STOCHF'] = talib.STOCHF(df['high'], df['low'], df['close'])
    #         new_columns['STOCHRSI'] = talib.STOCHRSI(df['close'])
    #         new_columns['MACDEXT'] = talib.MACDEXT(df['close'], fastperiod=12, fastmatype=0, slowperiod=26, slowmatype=0, signalperiod=9, signalmatype=0)
    #         new_columns['MACDFIX'] = talib.MACDFIX(df['close'], signalperiod=9)
    #########Volume Indicators
    new_columns['AD'] = talib.AD(df['high'], df['low'], df['close'], df['volume'])
    new_columns['ADOSC'] = talib.ADOSC(df['high'], df['low'], df['close'], df['volume'], fastperiod=3, slowperiod=10)
    new_columns['OBV'] = talib.OBV(df['close'], df['volume'])

    #########Cycle Indicators
    new_columns['HT_DCPERIOD'] = talib.HT_DCPERIOD(df['close'])
    new_columns['HT_DCPHASE'] = talib.HT_DCPHASE(df['close'])
    new_columns['HT_PHASOR_inphase'], new_columns['HT_PHASOR_quadrature'] = talib.HT_PHASOR(df['close'])
    # new_columns['HT_SINE'] = talib.HT_SINE(df['close'])
    new_columns['HT_TRENDMODE'] = talib.HT_TRENDMODE(df['close'])

    #########Price Transform
    new_columns['AVGPRICE'] = talib.AVGPRICE(df['open'], df['high'], df['low'], df['close'])
    new_columns['MEDPRICE'] = talib.MEDPRICE(df['high'], df['low'])
    new_columns['TYPPRICE'] = talib.TYPPRICE(df['high'], df['low'], df['close'])
    new_columns['WCLPRICE'] = talib.WCLPRICE(df['high'], df['low'], df['close'])
    #########Volatility Indicators
    new_columns['TRANGE'] = talib.TRANGE(df['high'], df['low'], df['close'])
    #########Pattern Recognition
    new_columns['CDL2CROWS'] = talib.CDL2CROWS(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDL3BLACKCROWS'] = talib.CDL3BLACKCROWS(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDL3INSIDE'] = talib.CDL3INSIDE(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDL3LINESTRIKE'] = talib.CDL3LINESTRIKE(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDL3OUTSIDE'] = talib.CDL3OUTSIDE(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDL3STARSINSOUTH'] = talib.CDL3STARSINSOUTH(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDL3WHITESOLDIERS'] = talib.CDL3WHITESOLDIERS(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLABANDONEDBABY'] = talib.CDLABANDONEDBABY(df['open'], df['high'], df['low'], df['close'], penetration=0)

    new_columns['CDLADVANCEBLOCK'] = talib.CDLADVANCEBLOCK(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLBELTHOLD'] = talib.CDLBELTHOLD(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLBREAKAWAY'] = talib.CDLBREAKAWAY(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLCLOSINGMARUBOZU'] = talib.CDLCLOSINGMARUBOZU(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLCONCEALBABYSWALL'] = talib.CDLCONCEALBABYSWALL(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLCOUNTERATTACK'] = talib.CDLCOUNTERATTACK(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLDARKCLOUDCOVER'] = talib.CDLDARKCLOUDCOVER(df['open'], df['high'], df['low'], df['close'], penetration=0)

    new_columns['CDLDOJI'] = talib.CDLDOJI(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLDOJISTAR'] = talib.CDLDOJISTAR(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLDRAGONFLYDOJI'] = talib.CDLDRAGONFLYDOJI(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLENGULFING'] = talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLEVENINGDOJISTAR'] = talib.CDLEVENINGDOJISTAR(df['open'], df['high'], df['low'], df['close'], penetration=0)

    new_columns['CDLEVENINGSTAR'] = talib.CDLEVENINGSTAR(df['open'], df['high'], df['low'], df['close'], penetration=0)
    new_columns['CDLGAPSIDESIDEWHITE'] = talib.CDLGAPSIDESIDEWHITE(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLGRAVESTONEDOJI'] = talib.CDLGRAVESTONEDOJI(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLHAMMER'] = talib.CDLHAMMER(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLHANGINGMAN'] = talib.CDLHANGINGMAN(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLHARAMI'] = talib.CDLHARAMI(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLHARAMICROSS'] = talib.CDLHARAMICROSS(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLHIGHWAVE'] = talib.CDLHIGHWAVE(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLHIKKAKE'] = talib.CDLHIKKAKE(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLHIKKAKEMOD'] = talib.CDLHIKKAKEMOD(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLHOMINGPIGEON'] = talib.CDLHOMINGPIGEON(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLIDENTICAL3CROWS'] = talib.CDLIDENTICAL3CROWS(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLINNECK'] = talib.CDLINNECK(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLINVERTEDHAMMER'] = talib.CDLINVERTEDHAMMER(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLKICKING'] = talib.CDLKICKING(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLKICKINGBYLENGTH'] = talib.CDLKICKINGBYLENGTH(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLLADDERBOTTOM'] = talib.CDLLADDERBOTTOM(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLLONGLEGGEDDOJI'] = talib.CDLLONGLEGGEDDOJI(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLLONGLINE'] = talib.CDLLONGLINE(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLMARUBOZU'] = talib.CDLMARUBOZU(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLMATCHINGLOW'] = talib.CDLMATCHINGLOW(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLMATHOLD'] = talib.CDLMATHOLD(df['open'], df['high'], df['low'], df['close'], penetration=0)
    new_columns['CDLMORNINGDOJISTAR'] = talib.CDLMORNINGDOJISTAR(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLMORNINGSTAR'] = talib.CDLMORNINGSTAR(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLONNECK'] = talib.CDLONNECK(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLPIERCING'] = talib.CDLPIERCING(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLRICKSHAWMAN'] = talib.CDLRICKSHAWMAN(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLRISEFALL3METHODS'] = talib.CDLRISEFALL3METHODS(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLSEPARATINGLINES'] = talib.CDLSEPARATINGLINES(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLSHOOTINGSTAR'] = talib.CDLSHOOTINGSTAR(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLSHORTLINE'] = talib.CDLSHORTLINE(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLSPINNINGTOP'] = talib.CDLSPINNINGTOP(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLSTALLEDPATTERN'] = talib.CDLSTALLEDPATTERN(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLSTICKSANDWICH'] = talib.CDLSTICKSANDWICH(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLTAKURI'] = talib.CDLTAKURI(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLTASUKIGAP'] = talib.CDLTASUKIGAP(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLTHRUSTING'] = talib.CDLTHRUSTING(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLTRISTAR'] = talib.CDLTRISTAR(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLUNIQUE3RIVER'] = talib.CDLUNIQUE3RIVER(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLUPSIDEGAP2CROWS'] = talib.CDLUPSIDEGAP2CROWS(df['open'], df['high'], df['low'], df['close'])
    new_columns['CDLXSIDEGAP3METHODS'] = talib.CDLXSIDEGAP3METHODS(df['open'], df['high'], df['low'], df['close'])
    #########Statistic Functions
    new_columns['LINEARREG'] = talib.LINEARREG(df['close'])
    new_columns['LINEARREG_ANGLE'] = talib.LINEARREG_ANGLE(df['close'])
    new_columns['LINEARREG_INTERCEPT'] = talib.LINEARREG_INTERCEPT(df['close'])
    new_columns['LINEARREG_SLOPE'] = talib.LINEARREG_SLOPE(df['close'])
    #         new_columns['STDDEV'] = df['close'].rolling(timeperiod).std()
    new_columns['TSF'] = talib.TSF(df['close'])
    new_columns['VAR'] = talib.VAR(df['close'])
    # Iterate over the time periods
    for timeperiod in master_dictionary["timeperiods"]:
        #########Overlap Studies
        new_columns[f'BB_upper_{timeperiod}'], new_columns[f'BB_middle_{timeperiod}'], new_columns[
            f'BB_lower_{timeperiod}'] = talib.BBANDS(df['close'], timeperiod=timeperiod)
        new_columns[f'DEMA_{timeperiod}'] = talib.DEMA(df['close'], timeperiod=timeperiod)
        new_columns[f'EMA_{timeperiod}'] = talib.EMA(df['close'], timeperiod=timeperiod)
        new_columns[f'KAMA_{timeperiod}'] = talib.KAMA(df['close'], timeperiod=timeperiod)
        new_columns[f'MA_{timeperiod}'] = talib.MA(df['close'], timeperiod=timeperiod)
        #         new_columns['MAMA'], new_columns['FAMA'] = talib.MAMA(df['close'], fastlimit=0, slowlimit=0)
        #         new_columns['MAVP'] = talib.MAVP(df['close'], periods=None, minperiod=2, maxperiod=30, matype=0)
        new_columns[f'MIDPOINT_{timeperiod}'] = talib.MIDPOINT(df['close'], timeperiod=timeperiod)
        new_columns[f'MIDPRICE_{timeperiod}'] = talib.MIDPRICE(df['high'], df['low'], timeperiod=timeperiod)
        new_columns[f'SMA_{timeperiod}'] = talib.SMA(df['close'], timeperiod=timeperiod)
        new_columns[f'TEMA_{timeperiod}'] = talib.TEMA(df['close'], timeperiod=timeperiod)
        new_columns[f'TRIMA_{timeperiod}'] = talib.TRIMA(df['close'], timeperiod=timeperiod)
        new_columns[f'WMA_{timeperiod}'] = talib.WMA(df['close'], timeperiod=timeperiod)
        new_columns[f'ADX_{timeperiod}'] = talib.ADX(df['high'], df['low'], df['close'], timeperiod=timeperiod)
        new_columns[f'ADXR_{timeperiod}'] = talib.ADXR(df['high'], df['low'], df['close'], timeperiod=timeperiod)
        new_columns[f'AROON_up_{timeperiod}'], new_columns[f'AROON_down_{timeperiod}'] = talib.AROON(df['high'], df['low'],
                                                                                   timeperiod=timeperiod)
        new_columns[f'AROONOSC_{timeperiod}'] = talib.AROONOSC(df['high'], df['low'], timeperiod=timeperiod)
        new_columns[f'CCI_{timeperiod}'] = talib.CCI(df['high'], df['low'], df['close'], timeperiod=timeperiod)
        new_columns[f'CMO_{timeperiod}'] = talib.CMO(df['close'], timeperiod=timeperiod)
        new_columns[f'DX_{timeperiod}'] = talib.DX(df['high'], df['low'], df['close'], timeperiod=timeperiod)
        new_columns[f'MFI_{timeperiod}'] = talib.MFI(df['high'], df['low'], df['close'], df['volume'],
                                            timeperiod=timeperiod)
        new_columns[f'MINUS_DI_{timeperiod}'] = talib.MINUS_DI(df['high'], df['low'], df['close'], timeperiod=timeperiod)
        new_columns[f'MINUS_DM_{timeperiod}'] = talib.MINUS_DM(df['high'], df['low'], timeperiod=timeperiod)
        new_columns[f'MOM_{timeperiod}'] = talib.MOM(df['close'], timeperiod=timeperiod)
        new_columns[f'PLUS_DI_{timeperiod}'] = talib.PLUS_DI(df['high'], df['low'], df['close'], timeperiod=timeperiod)
        new_columns[f'PLUS_DM_{timeperiod}'] = talib.PLUS_DM(df['high'], df['low'], timeperiod=timeperiod)
        new_columns[f'ROC_{timeperiod}'] = talib.ROC(df['close'], timeperiod=timeperiod)
        new_columns[f'ROCP_{timeperiod}'] = talib.ROCP(df['close'], timeperiod=timeperiod)
        new_columns[f'ROCR_{timeperiod}'] = talib.ROCR(df['close'], timeperiod=timeperiod)
        new_columns[f'ROCR100_{timeperiod}'] = talib.ROCR100(df['close'], timeperiod=timeperiod)
        new_columns[f'RSI_{timeperiod}'] = talib.RSI(df['close'], timeperiod=timeperiod)

        new_columns[f'ATR_{timeperiod}'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=timeperiod)
        new_columns[f'NATR_{timeperiod}'] = talib.NATR(df['high'], df['low'], df['close'], timeperiod=timeperiod)
        #########Statistic Functions
        new_columns[f'BETA_{timeperiod}'] = talib.BETA(df['high'], df['low'], timeperiod=timeperiod)
        new_columns[f'CORREL_{timeperiod}'] = talib.CORREL(df['high'], df['low'], timeperiod=timeperiod)
    # Save the updated dataframe to the CSV file
    df = pd.concat([df, new_columns], axis=1)
    display(df[500:].head())
    df_final = df[500:]
#     Append the data frame to the existing CSV file in append mode without headers
    df_final.to_csv(concatenated_file_path,
                    mode='a',
                    header=False,
                    index=False)

,open_time,open,high,low,close,volume,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,...,PLUS_DM_233,ROC_233,ROCP_233,ROCR_233,ROCR100_233,RSI_233,ATR_233,NATR_233,BETA_233,CORREL_233
7332,2023-05-11 21:30:00+05:30,27166.3,27214.4,26688.8,26831.4,146708.743,3.948798e+09,1463913,70334.441,1.893423e+09,...,20182.063565,-4.722100,-0.047221,0.952779,95.277900,49.900050,417.004749,1.554167,0.404240,0.958450
7333,2023-05-12 01:30:00+05:30,26831.5,27050.8,26787.3,26956.4,53500.320,1.441430e+09,652333,27390.346,7.380216e+08,...,20095.445266,-2.818145,-0.028181,0.971819,97.181855,50.049418,416.345930,1.544516,0.401827,0.958952
7334,2023-05-12 05:30:00+05:30,26956.4,27090.0,26515.0,26574.8,90812.315,2.428956e+09,905511,42437.631,1.135550e+09,...,20009.198720,-4.908307,-0.049083,0.950917,95.091693,49.596067,417.026849,1.569257,0.402353,0.959384
7335,2023-05-12 09:30:00+05:30,26574.7,26699.2,26100.1,26294.1,131694.685,3.469452e+09,1203311,61228.159,1.613222e+09,...,19923.322331,-6.901789,-0.069018,0.930982,93.098211,49.266390,417.808279,1.588981,0.403456,0.960271
7336,2023-05-12 13:30:00+05:30,26294.0,26425.8,26220.0,26333.0,54978.691,1.448095e+09,698087,27416.264,7.221716e+08,...,19837.814510,-6.062606,-0.060626,0.939374,93.937394,49.313284,416.898372,1.583178,0.399213,0.961062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7827,2023-08-02 09:30:00+05:30,29622.5,29719.5,29550.0,29570.1,45298.470,1.341810e+09,413134,23107.701,6.845590e+08,...,16193.104065,-3.601642,-0.036016,0.963984,96.398358,51.109860,311.458495,1.053289,0.244435,0.933305
7828,2023-08-02 13:30:00+05:30,29570.2,29571.0,29360.0,29552.9,75164.964,2.215702e+09,558340,36877.075,1.087107e+09,...,16123.605764,-2.757404,-0.027574,0.972426,97.242596,51.081694,311.027342,1.052443,0.249112,0.933729
7829,2023-08-02 17:30:00+05:30,29552.8,29599.0,29156.0,29312.6,97917.626,2.873900e+09,790262,46167.660,1.354984e+09,...,16054.405739,-4.276636,-0.042766,0.957234,95.723364,50.689742,311.593749,1.063003,0.248155,0.933864
7830,2023-08-02 21:30:00+05:30,29312.5,29357.0,28906.3,29141.8,115887.845,3.373807e+09,842066,54520.006,1.587364e+09,...,15985.502710,-4.528866,-0.045289,0.954711,95.471134,50.413610,312.190771,1.071282,0.254973,0.934163


500
setting up things only for incremental data for BTCUSDT4h


C:\Users\Admin\AppData\Local\Temp\ipykernel_21160\3249579248.py:50: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,2

,0,1,2,3,4,5,6,7,8,9,10
7333,2023-05-11 21:30:00+05:30,27166.3,27214.4,26688.8,26831.4,146708.743,3948797855.07034,1463913,70334.441,1893422951.4949,27166.3
7334,2023-05-12 01:30:00+05:30,26831.5,27050.8,26787.3,26956.4,53500.32,1441430159.0916,652333,27390.346,738021609.1936,26831.5
7335,2023-05-12 05:30:00+05:30,26956.4,27090.0,26515.0,26574.8,90812.315,2428955517.57565,905511,42437.631,1135549934.0871,26956.4
7336,2023-05-12 09:30:00+05:30,26574.7,26699.2,26100.1,26294.1,131694.685,3469452074.94754,1203311,61228.159,1613221931.2378,26574.7
7337,2023-05-12 13:30:00+05:30,26294.0,26425.8,26220.0,26333.0,54978.691,1448095313.2326,698087,27416.264,722171642.8684,26294.0
...,...,...,...,...,...,...,...,...,...,...,...
7828,2023-08-02 09:30:00+05:30,29622.5,29719.5,29550.0,29570.1,45298.47,1341809983.6361,413134,23107.701,684559035.7769,29622.5
7829,2023-08-02 13:30:00+05:30,29570.2,29571.0,29360.0,29552.9,75164.964,2215702366.78114,558340,36877.075,1087106520.4843,29570.2
7830,2023-08-02 17:30:00+05:30,29552.8,29599.0,29156.0,29312.6,97917.626,2873900205.03752,790262,46167.66,1354983655.4466,29552.8
7831,2023-08-02 21:30:00+05:30,29312.5,29357.0,28906.3,29141.8,115887.845,3373807201.57135,842066,54520.006,1587363910.69242,29312.5


0     object
1     object
2     object
3     object
4     object
5     object
6     object
7     object
8     object
9     object
10    object
dtype: object
01:30:00+0530
2023-08-02 00:00:00
2023-11-14 01:23:58.598665


...

C:\Users\Admin\AppData\Local\Temp\ipykernel_21160\3249579248.py:365: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[f'ADXR_{timeperiod}'] = talib.ADXR(df['high'], df['low'], df['close'], timeperiod=timeperiod)
C:\Users\Admin\AppData\Local\Temp\ipykernel_21160\3249579248.py:366: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[f'AROON_up_{timeperiod}'], new_columns[f'AROON_down_{timeperiod}'] = talib.AROON(df['high'], df['low'],
C:\Users\Admin\AppData\Local\Temp\ipykernel_21160\3249579248.py:366: PerformanceWa

C:\Users\Admin\AppData\Local\Temp\ipykernel_21160\3249579248.py:354: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[f'KAMA_{timeperiod}'] = talib.KAMA(df['close'], timeperiod=timeperiod)
C:\Users\Admin\AppData\Local\Temp\ipykernel_21160\3249579248.py:355: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[f'MA_{timeperiod}'] = talib.MA(df['close'], timeperiod=timeperiod)
C:\Users\Admin\AppData\Local\Temp\ipykernel_21160\3249579248.py:358: PerformanceWarning: DataFrame is highly fragmented.  This is usually the 

C:\Users\Admin\AppData\Local\Temp\ipykernel_21160\3249579248.py:388: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[f'BETA_{timeperiod}'] = talib.BETA(df['high'], df['low'], timeperiod=timeperiod)
C:\Users\Admin\AppData\Local\Temp\ipykernel_21160\3249579248.py:389: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[f'CORREL_{timeperiod}'] = talib.CORREL(df['high'], df['low'], timeperiod=timeperiod)
C:\Users\Admin\AppData\Local\Temp\ipykernel_21160\3249579248.py:350: PerformanceWarning: DataFrame is highly fragm

C:\Users\Admin\AppData\Local\Temp\ipykernel_21160\3249579248.py:372: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[f'MFI_{timeperiod}'] = talib.MFI(df['high'], df['low'], df['close'], df['volume'],
C:\Users\Admin\AppData\Local\Temp\ipykernel_21160\3249579248.py:374: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[f'MINUS_DI_{timeperiod}'] = talib.MINUS_DI(df['high'], df['low'], df['close'], timeperiod=timeperiod)
C:\Users\Admin\AppData\Local\Temp\ipykernel_21160\3249579248.py:375: PerformanceWarning: DataFr

,open_time,open,high,low,close,volume,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,...,PLUS_DM_233,ROC_233,ROCP_233,ROCR_233,ROCR100_233,RSI_233,ATR_233,NATR_233,BETA_233,CORREL_233
500,2023-08-03 05:30:00+05:30,29170.2,29273.1,29110.0,29170.1,24131.103,704256853.85220,256243,11679.432,340876039.81640,...,15225.379704,-5.026698,-0.050267,0.949733,94.973302,50.407048,297.492073,1.019853,0.247845,0.934794
501,2023-08-03 09:30:00+05:30,29170.1,29185.2,29005.0,29034.0,28883.917,840055251.22480,273728,13540.924,393853819.07450,...,15160.034727,-5.457506,-0.054575,0.945425,94.542494,50.175594,296.988673,1.022900,0.248331,0.935430
502,2023-08-03 13:30:00+05:30,29034.0,29193.5,28955.4,29134.2,39270.953,1142696470.31930,342160,19708.188,573542380.89750,...,15094.970200,-4.709542,-0.047095,0.952905,95.290458,50.344180,296.735932,1.018514,0.248020,0.935987
503,2023-08-03 17:30:00+05:30,29134.1,29440.0,29068.0,29244.4,78353.291,2291477941.10498,628480,39938.739,1168145602.59268,...,15276.684920,-3.864563,-0.038646,0.961354,96.135437,50.529069,297.058954,1.015781,0.248997,0.936200
504,2023-08-03 21:30:00+05:30,29244.4,29359.9,29119.3,29265.7,49258.672,1440824256.87480,413522,23495.098,687286452.66240,...,15211.119749,-3.914571,-0.039146,0.960854,96.085429,50.564800,296.816641,1.014213,0.247978,0.936608


In [43]:
import pandas as pd

df1 = pd.read_csv("D:\KISHORE\Binance-Data-Downloader\data\extracted_data\BTCUSDT-12h\BTCUSDT-12h_W0.73_L0.4_straight_download_v2.csv")
df2 = pd.read_csv("D:\KISHORE\Binance-Data-Downloader\data\extracted_data\BTCUSDT-12h\BTCUSDT-12h_W0.73_L0.4.csv")


In [90]:
dfs = [df1.head(200),df2.head(200)]
all(x.equals(dfs[0]) for x in dfs)

True

In [23]:
display(df1.head())
display(df2.head())

,open_time,open,high,low,close,volume,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,...,PLUS_DM_233,ROC_233,ROCP_233,ROCR_233,ROCR100_233,RSI_233,ATR_233,NATR_233,BETA_233,CORREL_233
0,2020-01-01 17:30:00+05:30,7192.74,7260.43,7174.00,7197.57,28970.926,2.091275e+08,48560,14798.826,1.068439e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-02 05:30:00+05:30,7197.57,7209.59,7101.00,7136.24,44326.072,3.167684e+08,99658,21591.765,1.543033e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-02 17:30:00+05:30,7136.36,7160.00,6922.00,6962.04,70969.605,4.988595e+08,125089,33812.497,2.376084e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-03 05:30:00+05:30,6962.34,7368.33,6863.44,7338.11,119991.949,8.594949e+08,233569,62408.336,4.470412e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-03 17:30:00+05:30,7338.40,7407.28,7229.00,7341.72,88501.509,6.478187e+08,176251,45077.629,3.300156e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,open_time,open,high,low,close,volume,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,...,PLUS_DM_233,ROC_233,ROCP_233,ROCR_233,ROCR100_233,RSI_233,ATR_233,NATR_233,BETA_233,CORREL_233
0,2020-01-01 17:30:00+05:30,7192.74,7260.43,7174.00,7197.57,28970.926,2.091275e+08,48560,14798.826,1.068439e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-02 05:30:00+05:30,7197.57,7209.59,7101.00,7136.24,44326.072,3.167684e+08,99658,21591.765,1.543033e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-02 17:30:00+05:30,7136.36,7160.00,6922.00,6962.04,70969.605,4.988595e+08,125089,33812.497,2.376084e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-03 05:30:00+05:30,6962.34,7368.33,6863.44,7338.11,119991.949,8.594949e+08,233569,62408.336,4.470412e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-03 17:30:00+05:30,7338.40,7407.28,7229.00,7341.72,88501.509,6.478187e+08,176251,45077.629,3.300156e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
# Check if the columns are equal
columns_equal = df1.columns.equals(df2.columns)

if columns_equal:
    # Check for equality of values in each corresponding cell
    values_equal = (df1 == df2).all().all()

    if values_equal:
        print("Both DataFrames are equal.")
    else:
        # Find differing cells
        differing_cells = (df1 != df2).stack()
        differing_cells = differing_cells[differing_cells].index

        print("Values differ in the following cells:")
        print(differing_cells)

else:
    print("Columns in both DataFrames are not equal.")

Values differ in the following cells:
MultiIndex([(   0, 'shorts_win_after'),
            (   0,  'longs_win_after'),
            (   0,   'entered_before'),
            (   0,     'HT_TRENDLINE'),
            (   0,              'SAR'),
            (   0,           'SAREXT'),
            (   0,               'T3'),
            (   0,              'APO'),
            (   0,             'MACD'),
            (   0,      'MACD_signal'),
            ...
            (8443,          'MFI_233'),
            (8443,     'MINUS_DI_233'),
            (8443,     'MINUS_DM_233'),
            (8443,      'PLUS_DI_233'),
            (8443,      'PLUS_DM_233'),
            (8443,          'RSI_233'),
            (8443,          'ATR_233'),
            (8443,         'NATR_233'),
            (8443,         'BETA_233'),
            (8443,       'CORREL_233')],
           length=229397)


In [9]:
display(df1.iloc[[7832]])
display(df2.iloc[[7832]])

,open_time,open,high,low,close,volume,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,entry,if_short,if_long,long_target,short_target,long_stop_loss,short_stop_loss,shorts_win_after,longs_win_after,dual_loss,entered_before,HT_TRENDLINE,SAR,SAREXT,T3,APO,BOP,MACD,MACD_signal,MACD_hist,PPO,TRIX,ULTOSC,WILLR,AD,ADOSC,OBV,HT_DCPERIOD,HT_DCPHASE,HT_PHASOR_inphase,HT_PHASOR_quadrature,HT_TRENDMODE,AVGPRICE,MEDPRICE,TYPPRICE,WCLPRICE,TRANGE,CDL2CROWS,CDL3BLACKCROWS,CDL3INSIDE,CDL3LINESTRIKE,CDL3OUTSIDE,CDL3STARSINSOUTH,CDL3WHITESOLDIERS,CDLABANDONEDBABY,CDLADVANCEBLOCK,CDLBELTHOLD,CDLBREAKAWAY,CDLCLOSINGMARUBOZU,CDLCONCEALBABYSWALL,CDLCOUNTERATTACK,CDLDARKCLOUDCOVER,CDLDOJI,CDLDOJISTAR,CDLDRAGONFLYDOJI,CDLENGULFING,CDLEVENINGDOJISTAR,CDLEVENINGSTAR,CDLGAPSIDESIDEWHITE,CDLGRAVESTONEDOJI,CDLHAMMER,CDLHANGINGMAN,CDLHARAMI,CDLHARAMICROSS,CDLHIGHWAVE,CDLHIKKAKE,CDLHIKKAKEMOD,CDLHOMINGPIGEON,CDLIDENTICAL3CROWS,CDLINNECK,CDLINVERTEDHAMMER,CDLKICKING,CDLKICKINGBYLENGTH,CDLLADDERBOTTOM,CDLLONGLEGGEDDOJI,CDLLONGLINE,CDLMARUBOZU,CDLMATCHINGLOW,CDLMATHOLD,CDLMORNINGDOJISTAR,CDLMORNINGSTAR,CDLONNECK,CDLPIERCING,CDLRICKSHAWMAN,CDLRISEFALL3METHODS,CDLSEPARATINGLINES,CDLSHOOTINGSTAR,CDLSHORTLINE,CDLSPINNINGTOP,CDLSTALLEDPATTERN,CDLSTICKSANDWICH,CDLTAKURI,CDLTASUKIGAP,CDLTHRUSTING,CDLTRISTAR,CDLUNIQUE3RIVER,CDLUPSIDEGAP2CROWS,CDLXSIDEGAP3METHODS,LINEARREG,LINEARREG_ANGLE,LINEARREG_INTERCEPT,LINEARREG_SLOPE,TSF,VAR,BB_upper_5,BB_middle_5,BB_lower_5,DEMA_5,EMA_5,KAMA_5,MA_5,MIDPOINT_5,MIDPRICE_5,SMA_5,TEMA_5,TRIMA_5,WMA_5,ADX_5,ADXR_5,AROON_up_5,AROON_down_5,AROONOSC_5,CCI_5,CMO_5,DX_5,MFI_5,MINUS_DI_5,MINUS_DM_5,MOM_5,PLUS_DI_5,PLUS_DM_5,ROC_5,ROCP_5,ROCR_5,ROCR100_5,RSI_5,ATR_5,NATR_5,BETA_5,CORREL_5,BB_upper_8,BB_middle_8,BB_lower_8,DEMA_8,EMA_8,KAMA_8,MA_8,MIDPOINT_8,MIDPRICE_8,SMA_8,TEMA_8,TRIMA_8,WMA_8,ADX_8,ADXR_8,AROON_up_8,AROON_down_8,AROONOSC_8,CCI_8,CMO_8,DX_8,MFI_8,MINUS_DI_8,MINUS_DM_8,MOM_8,PLUS_DI_8,PLUS_DM_8,ROC_8,ROCP_8,ROCR_8,ROCR100_8,RSI_8,ATR_8,NATR_8,BETA_8,CORREL_8,BB_upper_13,BB_middle_13,BB_lower_13,DEMA_13,EMA_13,KAMA_13,MA_13,MIDPOINT_13,MIDPRICE_13,SMA_13,TEMA_13,TRIMA_13,WMA_13,ADX_13,ADXR_13,AROON_up_13,AROON_down_13,AROONOSC_13,CCI_13,CMO_13,DX_13,MFI_13,MINUS_DI_13,MINUS_DM_13,MOM_13,PLUS_DI_13,PLUS_DM_13,ROC_13,ROCP_13,ROCR_13,ROCR100_13,RSI_13,ATR_13,NATR_13,BETA_13,CORREL_13,BB_upper_21,BB_middle_21,BB_lower_21,DEMA_21,EMA_21,KAMA_21,MA_21,MIDPOINT_21,MIDPRICE_21,SMA_21,TEMA_21,TRIMA_21,WMA_21,ADX_21,ADXR_21,AROON_up_21,AROON_down_21,AROONOSC_21,CCI_21,CMO_21,DX_21,MFI_21,MINUS_DI_21,MINUS_DM_21,MOM_21,PLUS_DI_21,PLUS_DM_21,ROC_21,ROCP_21,ROCR_21,ROCR100_21,RSI_21,ATR_21,NATR_21,BETA_21,CORREL_21,BB_upper_30,BB_middle_30,BB_lower_30,DEMA_30,EMA_30,KAMA_30,MA_30,MIDPOINT_30,MIDPRICE_30,SMA_30,TEMA_30,TRIMA_30,WMA_30,ADX_30,ADXR_30,AROON_up_30,AROON_down_30,AROONOSC_30,CCI_30,CMO_30,DX_30,MFI_30,MINUS_DI_30,MINUS_DM_30,MOM_30,PLUS_DI_30,PLUS_DM_30,ROC_30,ROCP_30,ROCR_30,ROCR100_30,RSI_30,ATR_30,NATR_30,BETA_30,CORREL_30,BB_upper_34,BB_middle_34,BB_lower_34,DEMA_34,EMA_34,KAMA_34,MA_34,MIDPOINT_34,MIDPRICE_34,SMA_34,TEMA_34,TRIMA_34,WMA_34,ADX_34,ADXR_34,AROON_up_34,AROON_down_34,AROONOSC_34,CCI_34,CMO_34,DX_34,MFI_34,MINUS_DI_34,MINUS_DM_34,MOM_34,PLUS_DI_34,PLUS_DM_34,ROC_34,ROCP_34,ROCR_34,ROCR100_34,RSI_34,ATR_34,NATR_34,BETA_34,CORREL_34,BB_upper_50,BB_middle_50,BB_lower_50,DEMA_50,EMA_50,KAMA_50,MA_50,MIDPOINT_50,MIDPRICE_50,SMA_50,TEMA_50,TRIMA_50,WMA_50,ADX_50,ADXR_50,AROON_up_50,AROON_down_50,AROONOSC_50,CCI_50,CMO_50,DX_50,MFI_50,MINUS_DI_50,MINUS_DM_50,MOM_50,PLUS_DI_50,PLUS_DM_50,ROC_50,ROCP_50,ROCR_50,ROCR100_50,RSI_50,ATR_50,NATR_50,BETA_50,CORREL_50,BB_upper_55,BB_middle_55,BB_lower_55,DEMA_55,EMA_55,KAMA_55,MA_55,MIDPOINT_55,MIDPRICE_55,SMA_55,TEMA_55,TRIMA_55,WMA_55,ADX_55,ADXR_55,AROON_up_55,AROON_down_55,AROONOSC_55,CCI_55,CMO_55,DX_55,MFI_55,MINUS_DI_55,MINUS_DM_55,MOM_55,PLUS_DI_55,PLUS_DM_55,ROC_55,ROCP_55,ROCR_55,ROCR100_55,RSI_55,ATR_55,NATR_55,BETA_55,CORREL_55,BB_upper_89,BB_middle_8

,open_time,open,high,low,close,volume,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,entry,if_short,if_long,long_target,short_target,long_stop_loss,short_stop_loss,shorts_win_after,longs_win_after,dual_loss,entered_before,HT_TRENDLINE,SAR,SAREXT,T3,APO,BOP,MACD,MACD_signal,MACD_hist,PPO,TRIX,ULTOSC,WILLR,AD,ADOSC,OBV,HT_DCPERIOD,HT_DCPHASE,HT_PHASOR_inphase,HT_PHASOR_quadrature,HT_TRENDMODE,AVGPRICE,MEDPRICE,TYPPRICE,WCLPRICE,TRANGE,CDL2CROWS,CDL3BLACKCROWS,CDL3INSIDE,CDL3LINESTRIKE,CDL3OUTSIDE,CDL3STARSINSOUTH,CDL3WHITESOLDIERS,CDLABANDONEDBABY,CDLADVANCEBLOCK,CDLBELTHOLD,CDLBREAKAWAY,CDLCLOSINGMARUBOZU,CDLCONCEALBABYSWALL,CDLCOUNTERATTACK,CDLDARKCLOUDCOVER,CDLDOJI,CDLDOJISTAR,CDLDRAGONFLYDOJI,CDLENGULFING,CDLEVENINGDOJISTAR,CDLEVENINGSTAR,CDLGAPSIDESIDEWHITE,CDLGRAVESTONEDOJI,CDLHAMMER,CDLHANGINGMAN,CDLHARAMI,CDLHARAMICROSS,CDLHIGHWAVE,CDLHIKKAKE,CDLHIKKAKEMOD,CDLHOMINGPIGEON,CDLIDENTICAL3CROWS,CDLINNECK,CDLINVERTEDHAMMER,CDLKICKING,CDLKICKINGBYLENGTH,CDLLADDERBOTTOM,CDLLONGLEGGEDDOJI,CDLLONGLINE,CDLMARUBOZU,CDLMATCHINGLOW,CDLMATHOLD,CDLMORNINGDOJISTAR,CDLMORNINGSTAR,CDLONNECK,CDLPIERCING,CDLRICKSHAWMAN,CDLRISEFALL3METHODS,CDLSEPARATINGLINES,CDLSHOOTINGSTAR,CDLSHORTLINE,CDLSPINNINGTOP,CDLSTALLEDPATTERN,CDLSTICKSANDWICH,CDLTAKURI,CDLTASUKIGAP,CDLTHRUSTING,CDLTRISTAR,CDLUNIQUE3RIVER,CDLUPSIDEGAP2CROWS,CDLXSIDEGAP3METHODS,LINEARREG,LINEARREG_ANGLE,LINEARREG_INTERCEPT,LINEARREG_SLOPE,TSF,VAR,BB_upper_5,BB_middle_5,BB_lower_5,DEMA_5,EMA_5,KAMA_5,MA_5,MIDPOINT_5,MIDPRICE_5,SMA_5,TEMA_5,TRIMA_5,WMA_5,ADX_5,ADXR_5,AROON_up_5,AROON_down_5,AROONOSC_5,CCI_5,CMO_5,DX_5,MFI_5,MINUS_DI_5,MINUS_DM_5,MOM_5,PLUS_DI_5,PLUS_DM_5,ROC_5,ROCP_5,ROCR_5,ROCR100_5,RSI_5,ATR_5,NATR_5,BETA_5,CORREL_5,BB_upper_8,BB_middle_8,BB_lower_8,DEMA_8,EMA_8,KAMA_8,MA_8,MIDPOINT_8,MIDPRICE_8,SMA_8,TEMA_8,TRIMA_8,WMA_8,ADX_8,ADXR_8,AROON_up_8,AROON_down_8,AROONOSC_8,CCI_8,CMO_8,DX_8,MFI_8,MINUS_DI_8,MINUS_DM_8,MOM_8,PLUS_DI_8,PLUS_DM_8,ROC_8,ROCP_8,ROCR_8,ROCR100_8,RSI_8,ATR_8,NATR_8,BETA_8,CORREL_8,BB_upper_13,BB_middle_13,BB_lower_13,DEMA_13,EMA_13,KAMA_13,MA_13,MIDPOINT_13,MIDPRICE_13,SMA_13,TEMA_13,TRIMA_13,WMA_13,ADX_13,ADXR_13,AROON_up_13,AROON_down_13,AROONOSC_13,CCI_13,CMO_13,DX_13,MFI_13,MINUS_DI_13,MINUS_DM_13,MOM_13,PLUS_DI_13,PLUS_DM_13,ROC_13,ROCP_13,ROCR_13,ROCR100_13,RSI_13,ATR_13,NATR_13,BETA_13,CORREL_13,BB_upper_21,BB_middle_21,BB_lower_21,DEMA_21,EMA_21,KAMA_21,MA_21,MIDPOINT_21,MIDPRICE_21,SMA_21,TEMA_21,TRIMA_21,WMA_21,ADX_21,ADXR_21,AROON_up_21,AROON_down_21,AROONOSC_21,CCI_21,CMO_21,DX_21,MFI_21,MINUS_DI_21,MINUS_DM_21,MOM_21,PLUS_DI_21,PLUS_DM_21,ROC_21,ROCP_21,ROCR_21,ROCR100_21,RSI_21,ATR_21,NATR_21,BETA_21,CORREL_21,BB_upper_30,BB_middle_30,BB_lower_30,DEMA_30,EMA_30,KAMA_30,MA_30,MIDPOINT_30,MIDPRICE_30,SMA_30,TEMA_30,TRIMA_30,WMA_30,ADX_30,ADXR_30,AROON_up_30,AROON_down_30,AROONOSC_30,CCI_30,CMO_30,DX_30,MFI_30,MINUS_DI_30,MINUS_DM_30,MOM_30,PLUS_DI_30,PLUS_DM_30,ROC_30,ROCP_30,ROCR_30,ROCR100_30,RSI_30,ATR_30,NATR_30,BETA_30,CORREL_30,BB_upper_34,BB_middle_34,BB_lower_34,DEMA_34,EMA_34,KAMA_34,MA_34,MIDPOINT_34,MIDPRICE_34,SMA_34,TEMA_34,TRIMA_34,WMA_34,ADX_34,ADXR_34,AROON_up_34,AROON_down_34,AROONOSC_34,CCI_34,CMO_34,DX_34,MFI_34,MINUS_DI_34,MINUS_DM_34,MOM_34,PLUS_DI_34,PLUS_DM_34,ROC_34,ROCP_34,ROCR_34,ROCR100_34,RSI_34,ATR_34,NATR_34,BETA_34,CORREL_34,BB_upper_50,BB_middle_50,BB_lower_50,DEMA_50,EMA_50,KAMA_50,MA_50,MIDPOINT_50,MIDPRICE_50,SMA_50,TEMA_50,TRIMA_50,WMA_50,ADX_50,ADXR_50,AROON_up_50,AROON_down_50,AROONOSC_50,CCI_50,CMO_50,DX_50,MFI_50,MINUS_DI_50,MINUS_DM_50,MOM_50,PLUS_DI_50,PLUS_DM_50,ROC_50,ROCP_50,ROCR_50,ROCR100_50,RSI_50,ATR_50,NATR_50,BETA_50,CORREL_50,BB_upper_55,BB_middle_55,BB_lower_55,DEMA_55,EMA_55,KAMA_55,MA_55,MIDPOINT_55,MIDPRICE_55,SMA_55,TEMA_55,TRIMA_55,WMA_55,ADX_55,ADXR_55,AROON_up_55,AROON_down_55,AROONOSC_55,CCI_55,CMO_55,DX_55,MFI_55,MINUS_DI_55,MINUS_DM_55,MOM_55,PLUS_DI_55,PLUS_DM_55,ROC_55,ROCP_55,ROCR_55,ROCR100_55,RSI_55,ATR_55,NATR_55,BETA_55,CORREL_55,BB_upper_89,BB_middle_8

In [44]:
for i in range(len(df1)):
    dfs = [df1.head(i),df2.head(i)]
    if all(x.equals(dfs[0]) for x in dfs):
        continue
    else:
        display(df1.iloc[[i]])
        break

,open_time,open,high,low,close,volume,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,...,PLUS_DM_233,ROC_233,ROCP_233,ROCR_233,ROCR100_233,RSI_233,ATR_233,NATR_233,BETA_233,CORREL_233
2715,2023-10-03 17:30:00+05:30,27549.1,27569.2,27123.0,27412.0,186665.254,5.108793e+09,1897214,91861.928,2.514356e+09,...,29289.346358,3.053406,0.030534,1.030534,103.053406,50.5215,567.921487,2.071799,0.052078,0.974297


In [10]:
differing_columns = df1.columns[(df1.iloc[7832] != df2.iloc[7832]) & ~(df1.iloc[7832].isna() & df2.iloc[7832].isna())]


if len(differing_columns) == 0:
    print("The values in both dataframes are identical.")
else:
    print("Columns with differing values:")
    print(differing_columns)

Columns with differing values:
Index(['SAR', 'APO', 'MACD_signal', 'MACD_hist', 'PPO', 'TRIX', 'ULTOSC', 'AD',
       'ADOSC', 'OBV',
       ...
       'MFI_233', 'MINUS_DI_233', 'MINUS_DM_233', 'PLUS_DI_233', 'PLUS_DM_233',
       'RSI_233', 'ATR_233', 'NATR_233', 'BETA_233', 'CORREL_233'],
      dtype='object', length=296)
